In [79]:
import os
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import json

corpus_name = "movie-corpus"
corpus = os.path.join("data", corpus_name)

def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "utterances.jsonl"))

b'{"id": "L1045", "conversation_id": "L1044", "text": "They do not!", "speaker": "u0", "meta": {"movie_id": "m0", "parsed": [{"rt": 1, "toks": [{"tok": "They", "tag": "PRP", "dep": "nsubj", "up": 1, "dn": []}, {"tok": "do", "tag": "VBP", "dep": "ROOT", "dn": [0, 2, 3]}, {"tok": "not", "tag": "RB", "dep": "neg", "up": 1, "dn": []}, {"tok": "!", "tag": ".", "dep": "punct", "up": 1, "dn": []}]}]}, "reply-to": "L1044", "timestamp": null, "vectors": []}\n'
b'{"id": "L1044", "conversation_id": "L1044", "text": "They do to!", "speaker": "u2", "meta": {"movie_id": "m0", "parsed": [{"rt": 1, "toks": [{"tok": "They", "tag": "PRP", "dep": "nsubj", "up": 1, "dn": []}, {"tok": "do", "tag": "VBP", "dep": "ROOT", "dn": [0, 2, 3]}, {"tok": "to", "tag": "TO", "dep": "dobj", "up": 1, "dn": []}, {"tok": "!", "tag": ".", "dep": "punct", "up": 1, "dn": []}]}]}, "reply-to": null, "timestamp": null, "vectors": []}\n'
b'{"id": "L985", "conversation_id": "L984", "text": "I hope so.", "speaker": "u0", "meta": {

In [80]:
# Splits each line of the file to create lines and conversations
def loadLinesAndConversations(fileName, n):
    lines = {}
    conversations = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            lineJson = json.loads(line)
            # Extract fields for line object
            lineObj = {}
            lineObj["lineID"] = lineJson["id"] #example : L1045
            lineObj["characterID"] = lineJson["speaker"] # example : u1, u2
            lineObj["text"] = lineJson["text"] #These are the conversations
            lines[lineObj['lineID']] = lineObj

            # Extract fields for conversation object
            if lineJson["conversation_id"] not in conversations:
                convObj = {}
                convObj["conversationID"] = lineJson["conversation_id"]
                convObj["movieID"] = lineJson["meta"]["movie_id"]
                convObj["lines"] = [lineObj]
            else:
                convObj = conversations[lineJson["conversation_id"]]
                convObj["lines"].insert(0, lineObj)
            conversations[convObj["conversationID"]] = convObj
    first_n_items = dict(itertools.islice(lines.items(), n))
    print(first_n_items)
    print("Line dictionary ends!! \n")
    second_n_items = dict(itertools.islice(conversations.items(), n))
    print(second_n_items)

    return lines, conversations


# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations.values():
        # Iterate over all the lines of the conversation
        for i in range(len(conversation["lines"]) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

In [81]:
import codecs
import itertools
# Define path to new file
datafile = os.path.join(corpus, "formatted_movie_lines.txt")

delimiter = ','
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# Initialize lines dict and conversations dict
lines = {}
conversations = {}
# Load lines and conversations
print("\nProcessing corpus into lines and conversations...")
lines, conversations = loadLinesAndConversations(os.path.join(corpus, "utterances.jsonl"), 10)

# Write new csv file
print("\nWriting newly formatted file...")
# with open(datafile, 'w', encoding='utf-8') as outputfile:
#     writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
#     for pair in extractSentencePairs(conversations):
#         writer.writerow(pair)

# # Print a sample of lines
# print("\nSample lines from file:")
# printLines(datafile)


Processing corpus into lines and conversations...
{'L1045': {'lineID': 'L1045', 'characterID': 'u0', 'text': 'They do not!'}, 'L1044': {'lineID': 'L1044', 'characterID': 'u2', 'text': 'They do to!'}, 'L985': {'lineID': 'L985', 'characterID': 'u0', 'text': 'I hope so.'}, 'L984': {'lineID': 'L984', 'characterID': 'u2', 'text': 'She okay?'}, 'L925': {'lineID': 'L925', 'characterID': 'u0', 'text': "Let's go."}, 'L924': {'lineID': 'L924', 'characterID': 'u2', 'text': 'Wow'}, 'L872': {'lineID': 'L872', 'characterID': 'u0', 'text': "Okay -- you're gonna need to learn how to lie."}, 'L871': {'lineID': 'L871', 'characterID': 'u2', 'text': 'No'}, 'L870': {'lineID': 'L870', 'characterID': 'u0', 'text': 'I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?'}, 'L869': {'lineID': 'L869', 'characterID': 'u0', 'text': 'Like my fear of wearing pastels?'}}
Line dictionary ends!! 

{'L1044': {'conversationID': 'L1044', 'movieID': 'm0', 'lines': [{'lineI

In [4]:
!pip install pandas

In [82]:
import pandas as pd
df = pd.read_csv("./data/movie-corpus/formatted_movie_lines.txt")
# Define the text to add
import re
def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r"i'm", "i am", txt)
    txt = re.sub(r"he's", "he is", txt)
    txt = re.sub(r"she's", "she is", txt)
    txt = re.sub(r"that's", "that is", txt)
    txt = re.sub(r"what's", "what is", txt)
    txt = re.sub(r"where's", "where is", txt)
    txt = re.sub(r"\'ll", " will", txt)
    txt = re.sub(r"\'ve", " have", txt)
    txt = re.sub(r"\'re", " are", txt)
    txt = re.sub(r"\'d", " would", txt)
    txt = re.sub(r"won't", "will not", txt)
    txt = re.sub(r"can't", "can not", txt)
    txt = re.sub(r"[^\w\s]", "", txt)
    return txt
df['Input'] = df['Input'].apply(lambda x: clean_text(x))
df['Output'] = df['Output'].apply(lambda x: clean_text(x))


In [83]:
start_text = '<SOS> '
end_text = ' <EOS>'

# Add text to the specific column 'A'
df['Input'] = df['Input'].apply(lambda x: f"{start_text }{x}{ end_text}")
df['Output'] = df['Output'].apply(lambda x: f"{start_text }{x}{ end_text}")
df

,Input,Output
0,<SOS> they do to <EOS>,<SOS> they do not <EOS>
1,<SOS> she okay <EOS>,<SOS> i hope so <EOS>
2,<SOS> wow <EOS>,<SOS> lets go <EOS>
3,<SOS> i am kidding you know how sometimes you...,<SOS> no <EOS>
4,<SOS> no <EOS>,<SOS> okay you are gonna need to learn how to...
...,...,...
221277,<SOS> and i assure you you do not in fact i wo...,<SOS> so far only their scouts but we have had...
221278,<SOS> your orders mr vereker <EOS>,<SOS> i am to take the sikali with the main co...
221279,<SOS> i am to take the sikali with the main co...,<SOS> lord chelmsford seems to want me to stay...
221280,<SOS> lord chelmsford seems to want me to stay...,<SOS> i think chelmsford wants a good man on t...


In [84]:
df_copy = df

In [85]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221282 entries, 0 to 221281
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Input   221282 non-null  object
 1   Output  221282 non-null  object
dtypes: object(2)
memory usage: 3.4+ MB


In [91]:
# Filter rows where the length of split words in each column is <= 10
filtered_df = df_copy[df_copy.apply(lambda row: all(len(col.split()) <= 12 for col in row), axis=1)].reset_index()
filtered_df

,index,Input,Output
0,0,<SOS> they do to <EOS>,<SOS> they do not <EOS>
1,1,<SOS> she okay <EOS>,<SOS> i hope so <EOS>
2,2,<SOS> wow <EOS>,<SOS> lets go <EOS>
3,4,<SOS> no <EOS>,<SOS> okay you are gonna need to learn how to...
4,5,<SOS> i figured you would get to the good stuf...,<SOS> what good stuff <EOS>
...,...,...,...
98723,221265,<SOS> stuart <EOS>,<SOS> yes <EOS>
98724,221266,<SOS> yes <EOS>,<SOS> how quickly can you move your artillery ...
98725,221269,<SOS> well fed or hungry pulleine wants them i...,<SOS> right bombardier to me please <EOS>
98726,221271,<SOS> do you think she might be interested in ...,<SOS> which one <EOS>


In [8]:
import gensim

In [88]:
# Function to preprocess while preserving <SOS> and <EOS>
def preprocess_with_tags(text):
    words = text.split()
    processed_words = []
    for word in words:
        if word in ["<SOS>", "<EOS>"]:
            processed_words.append(word)
        else:
            processed_words.extend(gensim.utils.simple_preprocess(word))
    return processed_words

# Sample text
review_text = "<SOS> they do to <EOS>"

# Preprocess the text
processed_text = preprocess_with_tags(review_text)

print(processed_text)

['<SOS>', 'they', 'do', 'to', '<EOS>']


In [92]:
new_column = pd.concat([filtered_df['Input'], filtered_df['Output']], ignore_index=True)
new_column

0                                    <SOS> they do to <EOS>
1                                      <SOS> she okay <EOS>
2                                           <SOS> wow <EOS>
3                                            <SOS> no <EOS>
4         <SOS> i figured you would get to the good stuf...
                                ...                        
197451                                      <SOS> yes <EOS>
197452    <SOS> how quickly can you move your artillery ...
197453           <SOS> right  bombardier to me please <EOS>
197454                                <SOS> which one <EOS>
197455    <SOS> well that one the one who keeps looking ...
Length: 197456, dtype: object

In [94]:
count = 0
max = 0
for i in range(len(new_column)):
    count = len(new_column[i].split())
    if count > max:
        max = count
    count = 0
print(max)

12


In [95]:
review_text = new_column.apply(lambda x: preprocess_with_tags(x))

In [96]:
type(review_text)

pandas.core.series.Series

In [97]:
review_text.loc[192]

['<SOS>', 'say', 'do', 'what', 'you', 'wanna', 'do', '<EOS>']

In [98]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=4,
    workers=4,
)
model.build_vocab(review_text, progress_per=1000)
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)
vocab_size = len(model.wv.key_to_index)
print("Vocabulary size:", vocab_size)

Vocabulary size: 9525


In [99]:
def search_word_vector(model, word):
    if word in model.wv.key_to_index:
        vector = model.wv[word]
        print(f"Vector for the word '{word}':\n{vector}")
    else:
        print(f"The word '{word}' is not in the vocabulary.")

search_word_vector(model, "morning")

Vector for the word 'morning':
[-0.12011132  0.384622   -0.15799433  0.3210568   0.49736395 -0.5900213
 -0.08336844  0.87894773 -1.2149109   0.4858342  -0.21717048 -1.1692823
 -0.62061596  1.0418295  -0.25395197 -0.5788785   0.02252335  0.19325288
 -0.0026791  -0.8923486   0.67126286  0.09370332  0.9022529  -0.23918276
  0.87901676 -0.10139964  0.2430153   0.3173359  -0.63683474 -0.19720043
  0.7854393   0.24688905  0.03222169  0.25488296 -0.04853523  0.07995187
  0.71817917 -1.0489568  -0.37199947 -0.8017701  -0.3987603  -0.31087512
 -0.8043076   0.18803518 -0.07598886  1.2383559   0.3982284   0.815971
  0.44939354 -0.62986994  0.27540803  0.66275     0.5839314  -0.54130524
 -0.80799675 -0.41315117  0.42124674 -0.4756348  -0.65298754  0.37842572
  0.561582   -0.20618445 -0.6149973   0.5399425  -0.78569937 -0.9815398
  0.2246714   0.7103429  -0.1590571   0.13897036  0.96357584 -0.42784545
  0.01243685  0.11778596  1.141583    1.2254637  -0.34182286  0.2271248
 -0.16988152  0.18186165 -

In [100]:
model.wv.most_similar("morning")

[('afternoon', 0.8214861750602722),
 ('evening', 0.7791115641593933),
 ('weekend', 0.7235478162765503),
 ('night', 0.7098230123519897),
 ('content', 0.6578639149665833),
 ('luck', 0.6557112336158752),
 ('day', 0.6201649308204651),
 ('year', 0.617409348487854),
 ('drugstore', 0.6165878176689148),
 ('week', 0.6120112538337708)]

In [101]:
# Initialize an empty dictionary
word_vectors = {}

# Iterate through the vocabulary
for word in model.wv.index_to_key:
    # Add the word and its vector to the dictionary
    #print(type( model.wv[word]))
    word_vectors[word] = model.wv[word]

# Now word_vectors is a dictionary with words as keys and vectors as values
top_three_word_vectors = dict(list(word_vectors.items())[:3])
print(top_three_word_vectors)

{'<SOS>': array([ 1.1115055 , -0.2854384 ,  0.2021238 , -0.6217669 , -0.6933692 ,
       -0.8168397 , -0.16299292,  0.740368  , -0.12952244,  0.18133585,
        0.6142206 , -0.28838816,  0.12359715,  0.22400317,  0.9092182 ,
       -0.42435244, -0.42218113, -0.36179215,  0.31091332, -1.3080089 ,
        1.4772648 , -0.05265462, -0.7668279 ,  0.34306273,  0.3530216 ,
        0.1884358 ,  0.67026126,  1.0856019 , -0.7882152 ,  0.18373744,
       -0.68636966,  0.31173658,  0.27774388, -0.22424884, -0.23144087,
        0.3955155 ,  1.3653605 , -0.28003174, -0.46463773, -1.3446448 ,
       -1.5426947 ,  0.05352983, -0.25557604, -0.6083303 , -0.9108917 ,
        0.03340705, -0.37861347,  0.16701734,  0.03465249,  0.5073217 ,
       -0.22328208, -0.64848864, -1.4312875 , -0.7043642 ,  0.6696109 ,
       -0.6709416 ,  1.0156107 , -1.5408536 , -0.8795676 , -0.7562373 ,
       -0.3392753 ,  0.5410064 , -0.44258037,  0.8056419 , -1.1505798 ,
        0.30592555,  0.01169863,  0.05931721, -0.49311

In [102]:
import numpy as np
key1 = '<OUT>'
value1 = [1] * 100
key2 = '<PAD>'
value2 = [0] * 100
word_vectors[key1] = np.array(value1, dtype=np.float32)
word_vectors[key2] = np.array(value2, dtype=np.float32)

In [103]:
for key, values in word_vectors.items():
    if key == '<OUT>' or key == '<PAD>':
        print(key)
        print(values)

<OUT>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
<PAD>
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [104]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
#from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the pre-trained Word2Vec model
#model = Word2Vec.load("./data/movie-corpus/word2vec.model")

# Convert sentences to sequences of vectors

def sentence_to_vectors(sentence):
    gensim_sentences_list = preprocess_with_tags(sentence)
    #vectors = [model.wv[word] for word in gensim_sentences_list if word in model.wv]
#     vectors = [word_vectors[word] for word in word_vectors.keys() if word in wor]
    global vectors
    vectors= []
    for word in gensim_sentences_list:
        if word in word_vectors.keys():
            vectors.append(word_vectors[word])
        else:
            vectors.append(word_vectors['<OUT>'])
        
        
#     for word in sentence.lower.split():
#         if word.contains("'")
    return vectors

In [107]:
filtered_df['input_vectors'] = filtered_df['Input'].apply(sentence_to_vectors)
filtered_df['target_vectors'] = filtered_df['Output'].apply(sentence_to_vectors)

In [108]:
filtered_df['input_vectors']

0        [[1.1115055, -0.2854384, 0.2021238, -0.6217669...
1        [[1.1115055, -0.2854384, 0.2021238, -0.6217669...
2        [[1.1115055, -0.2854384, 0.2021238, -0.6217669...
3        [[1.1115055, -0.2854384, 0.2021238, -0.6217669...
4        [[1.1115055, -0.2854384, 0.2021238, -0.6217669...
                               ...                        
98723    [[1.1115055, -0.2854384, 0.2021238, -0.6217669...
98724    [[1.1115055, -0.2854384, 0.2021238, -0.6217669...
98725    [[1.1115055, -0.2854384, 0.2021238, -0.6217669...
98726    [[1.1115055, -0.2854384, 0.2021238, -0.6217669...
98727    [[1.1115055, -0.2854384, 0.2021238, -0.6217669...
Name: input_vectors, Length: 98728, dtype: object

In [112]:
#input_vectors = df_copy['input_vectors'].tolist()
#target_vectors = df_copy['target_vectors'].tolist()

In [121]:
def padding_fn(data):
    for i in data:
        while len(i) != 12:
            i.append(word_vectors['<PAD>'])
padding_fn(filtered_df['input_vectors'])
padding_fn(filtered_df['target_vectors'])

In [125]:
X = torch.tensor(filtered_df['input_vectors'], dtype = torch.float32)
X.shape

torch.Size([98728, 12, 100])

In [128]:
y = torch.tensor(filtered_df['target_vectors'], dtype = torch.float32)
y.shape

torch.Size([98728, 12, 100])

In [130]:
X_train= X[:78982]
y_train= y[:78982]
X_test = X[78983:]
y_test = y[78983:]

In [132]:
X_train_shape = X_train.shape
y_train_shape = y_train.shape
X_test_shape = X_test.shape
y_test_shape = y_test.shape
print(f"Shape of X train: {X_train_shape}")
print(f"Shape of y train: {y_train_shape}")
print(f"Shape of X test: {X_test_shape}")
print(f"Shape of y test: {y_test_shape}")

Shape of X train: torch.Size([78982, 12, 100])
Shape of y train: torch.Size([78982, 12, 100])
Shape of X test: torch.Size([19745, 12, 100])
Shape of y test: torch.Size([19745, 12, 100])


In [40]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        return hidden, cell


In [41]:
import torch
import torch.nn as nn

class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim + hidden_dim, hidden_dim)
        self.fc_out = nn.Linear(embedding_dim + hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell, context):
        # input = [batch size]
        # hidden = [1, batch size, hidden dim]
        # cell = [1, batch size, hidden dim]
        # context = [1, batch size, hidden dim]
        
        input = input.unsqueeze(0)
        # input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        
        emb_con = torch.cat((embedded, context), dim=2)
        # emb_con = [1, batch size, embedding dim + hidden dim]
        
        output, (hidden, cell) = self.rnn(emb_con, (hidden, cell))
        # output = [1, batch size, hidden dim]
        # hidden = [1, batch size, hidden dim]
        # cell = [1, batch size, hidden dim]
        
        output = torch.cat(
            (embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), dim=1
        )
        # output = [batch size, embedding dim + hidden dim * 2]
        
        prediction = self.fc_out(output)
        # prediction = [batch size, output dim]
        
        return prediction, hidden, cell


In [42]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is the context
        context = self.encoder(src)
        # context = [n layers * n directions, batch size, hidden dim]
        # context also used as the initial hidden state of the decoder
        hidden = context
        # hidden = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden state and the context state
            # receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            # output = [batch size, output dim]
            # hidden = [1, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

In [44]:
input_dim = 100
output_dim = 100
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    decoder_dropout,
)

model_0 = Seq2Seq(encoder, decoder, device).to(device)

In [45]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)


model_0.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(100, 256)
    (rnn): LSTM(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(100, 256)
    (rnn): LSTM(768, 512)
    (fc_out): Linear(in_features=1280, out_features=100, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [46]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 4,381,796 trainable parameters


In [49]:
optimizer = torch.optim.Adam(model.parameters())

In [52]:
loss_fn = nn.CrossEntropyLoss(ignore_index=word_vectors['<PAD>'])

In [ ]:
torch.manual_seed(42)

# Set the number of epochs
epochs = 100

# Put data to target device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

# Build training and evaluation loop
for epoch in range(epochs):
    ### Training
    model_0.train()

    # 1. Forward pass (model outputs raw logits)
    y_logits = model_0(X_train).squeeze() # squeeze to remove extra `1` dimensions, this won't work unless model and data are on same device 
    y_pred = torch.round(torch.sigmoid(y_logits)) # turn logits -> pred probs -> pred labls
  
    # 2. Calculate loss/accuracy
    # loss = loss_fn(torch.sigmoid(y_logits), # Using nn.BCELoss you need torch.sigmoid()
    #                y_train) 
    loss = loss_fn(y_logits, # Using nn.BCEWithLogitsLoss works with raw logits
                   y_train) 
    acc = accuracy_fn(y_true=y_train, 
                      y_pred=y_pred) 

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model_0.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_logits = model_0(X_test).squeeze() 
        test_pred = torch.round(torch.sigmoid(test_logits))
        # 2. Caculate loss/accuracy
        test_loss = loss_fn(test_logits,
                            y_test)
        test_acc = accuracy_fn(y_true=y_test,
                               y_pred=test_pred)

    # Print out what's happening every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")
    
    
